ÉTAPE 2 — Lecture du FASTA avec Biopython

In [7]:
from Bio import SeqIO

fasta_path = "data/example.fasta"

for i, record in enumerate(SeqIO.parse(fasta_path, "fasta"), 1):
    print(f"Séquence {i}")
    print(f"  ID : {record.id}")
    print(f"  Description : {record.description}")
    print(f"  Longueur : {len(record.seq)} bp")
    print("-" * 40)

Séquence 1
  ID : seq1
  Description : seq1 Homo_sapiens_test
  Longueur : 120 bp
----------------------------------------
Séquence 2
  ID : seq2
  Description : seq2 Mus_musculus_test
  Longueur : 120 bp
----------------------------------------


ÉTAPE 3 — Afficher les informations complètes d’une séquence

In [9]:
record = next(SeqIO.parse(fasta_path, "fasta"))

print("ID :", record.id)
print("Description :", record.description)
print("Séquence :")
print(record.seq)

ID : seq1
Description : seq1 Homo_sapiens_test
Séquence :
ATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC
Protéine : ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('MLAS*LAS*LAS*LAS*LASMLAS*LAS*LAS*LAS*LAS')


ÉTAPE 4 — Manipulation de la séquence ADN

In [10]:
from Bio.Seq import Seq

dna_seq = record.seq

print("ADN :", dna_seq)
print("Complément inverse :", dna_seq.reverse_complement())
print("ARN :", dna_seq.transcribe())
print("Protéine :", dna_seq.translate())

ADN : ATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC
Complément inverse : GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCAT
ARN : AUGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCAUGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGCUAGC
Protéine : MLAS*LAS*LAS*LAS*LASMLAS*LAS*LAS*LAS*LAS


ÉTAPE 5 — Validation des séquences

In [20]:
def clean_sequence(seq):
    return seq.replace(" ", "").replace("\n", "").upper()

def has_valid_characters(seq):
    return set(seq).issubset({"A", "C", "G", "T", "N"})

def is_long_enough(seq, min_length=100):
    return len(seq) >= min_length

fasta_path = "data/example.fasta"

valid_sequences = []
invalid_sequences = []
reasons = {}

for record in SeqIO.parse(fasta_path, "fasta"):
    seq = clean_sequence(str(record.seq))
    reason = None
    
    if not has_valid_characters(seq):
        reason = "Caractères non autorisés"
    elif not is_long_enough(seq):
        reason = "Séquence trop courte (<100 nucléotides)"
    
    if reason is None:
        valid_sequences.append(record)
        valid_count += 1
    else:
        invalid_sequences.append((record.id, reason))
        invalid_count += 1
        reasons[reason] = reasons.get(reason, 0) + 1

print("Séquences valides :", valid_count)
print("Séquences invalides :", invalid_count)
print("\nRaisons de rejet :")
for r, n in reasons.items():
    print(f"- {r} : {n}")

# Afficher les séquences rejetées
if invalid_sequences:
    print("\nDétail des séquences rejetées :")
    for seq_id, reason in invalid_sequences:
        print(f"  ✗ {seq_id} : {reason}")

Séquences valides : 6
Séquences invalides : 0

Raisons de rejet :


ÉTAPE 6 — Calcul des statistiques : longueur, comptage A/C/G/T, %GC

In [12]:
def calculer_statistiques(seq_str):
    """Calcule les statistiques d'une séquence."""
    longueur = len(seq_str)
    
    stats = {
        "longueur": longueur,
        "A": seq_str.count('A'),
        "C": seq_str.count('C'),
        "G": seq_str.count('G'),
        "T": seq_str.count('T')
    }
    
    # Calcul %GC
    stats["GC%"] = ((stats['G'] + stats['C']) / longueur * 100) if longueur > 0 else 0
    
    return stats

# Utilisation
for record in SeqIO.parse(fasta_path, "fasta"):
    seq = clean_sequence(str(record.seq))
    stats = calculer_statistiques(seq)
    
    print(f"Séquence : {record.id}")
    for key, value in stats.items():
        if key == "GC%":
            print(f"  {key} : {value:.2f}%")
        else:
            print(f"  {key} : {value}")
    print("-" * 40)

Séquence : seq1
  longueur : 120
  A : 30
  C : 30
  G : 30
  T : 30
  GC% : 50.00%
----------------------------------------
Séquence : seq2
  longueur : 120
  A : 29
  C : 29
  G : 31
  T : 31
  GC% : 50.00%
----------------------------------------


ÉTAPE 7 — Création de fonctions réutilisables

In [17]:

print("\n" + "="*60)
print("QUESTION 9: FONCTIONS RÉUTILISABLES")
print("="*60)

# Fonction 1: Charger FASTA
def charger_fasta(fasta_path):
    """Charge un fichier FASTA et retourne une liste de séquences."""
    sequences = []
    for record in SeqIO.parse(fasta_path, "fasta"):
        sequences.append(record)
    return sequences

# Fonction 2: Calculer statistiques
def statistiques_sequence(sequence):
    """Calcule les statistiques d'une séquence."""
    seq_str = str(sequence).upper()
    longueur = len(seq_str)
    gc_content = ((seq_str.count('G') + seq_str.count('C')) / longueur * 100) if longueur > 0 else 0
    
    return {
        "longueur": longueur,
        "A": seq_str.count('A'),
        "C": seq_str.count('C'),
        "G": seq_str.count('G'),
        "T": seq_str.count('T'),
        "GC%": round(gc_content, 2)
    }

mes_sequences = charger_fasta("data/example.fasta")

print(f"✓ {len(mes_sequences)} séquences chargées")

for seq in mes_sequences:
    stats = statistiques_sequence(seq.seq)
    print(f"\n{seq.id}: {stats}")


QUESTION 9: FONCTIONS RÉUTILISABLES
✓ 2 séquences chargées

seq1: {'longueur': 120, 'A': 30, 'C': 30, 'G': 30, 'T': 30, 'GC%': 50.0}

seq2: {'longueur': 120, 'A': 29, 'C': 29, 'G': 31, 'T': 31, 'GC%': 50.0}


ÉTAPE 8 — Classe SequenceItem avec POO

In [18]:
class SequenceItem:
    """Classe représentant une séquence biologique avec ses métadonnées."""
    
    def __init__(self, identifiant, description, sequence):
        """
        Initialise un objet SequenceItem.
        
        Args:
            identifiant (str): Identifiant de la séquence
            description (str): Description complète
            sequence (Seq ou str): Séquence biologique
        """
        self._identifiant = identifiant
        self._description = description
        self._sequence = Seq(sequence) if isinstance(sequence, str) else sequence
    
    # Accesseurs (getters) avec @property
    @property
    def identifiant(self):
        """Retourne l'identifiant de la séquence."""
        return self._identifiant
    
    @property
    def description(self):
        """Retourne la description de la séquence."""
        return self._description
    
    @property
    def sequence(self):
        """Retourne la séquence."""
        return self._sequence
    
    # Setter pour la séquence (optionnel)
    @sequence.setter
    def sequence(self, value):
        """Modifie la séquence."""
        self._sequence = Seq(value) if isinstance(value, str) else value
    
    # Question 11 : Méthodes d'analyse
    def length(self):
        """Retourne la longueur de la séquence."""
        return len(self._sequence)
    
    def gc_content(self):
        """Retourne le pourcentage GC."""
        seq_str = str(self._sequence).upper()
        if len(seq_str) == 0:
            return 0
        gc_count = seq_str.count('G') + seq_str.count('C')
        return (gc_count / len(seq_str)) * 100
    
    def reverse_complement(self):
        """Retourne le complément inverse de la séquence."""
        return self._sequence.reverse_complement()
    
    # Méthode spéciale pour affichage
    def __str__(self):
        """Représentation en chaîne de l'objet."""
        return f"SequenceItem(id={self._identifiant}, length={self.length()})"
    
    def __repr__(self):
        """Représentation pour le debug."""
        return self.__str__()


# =============================================================================
# UTILISATION DE LA CLASSE SequenceItem
# =============================================================================
print("\n--- Création d'objets SequenceItem ---\n")

# Exemple 1 : Créer une séquence manuellement
seq1 = SequenceItem(
    identifiant="seq_test",
    description="Séquence de test Homo sapiens",
    sequence="ATGCTAGCTAGCTAGCGCGC"
)

print(f"Objet créé : {seq1}")
print(f"  Identifiant : {seq1.identifiant}")
print(f"  Description : {seq1.description}")
print(f"  Longueur : {seq1.length()} nucléotides")
print(f"  %GC : {seq1.gc_content():.2f}%")
print(f"  Complément inverse : {seq1.reverse_complement()}")
print("-" * 40)

# Exemple 2 : Créer des objets depuis le fichier FASTA
print("\n--- Chargement depuis FASTA ---\n")

from Bio import SeqIO

fasta_path = "data/example.fasta"
sequences_objets = []

for record in SeqIO.parse(fasta_path, "fasta"):
    seq_item = SequenceItem(
        identifiant=record.id,
        description=record.description,
        sequence=record.seq
    )
    sequences_objets.append(seq_item)

print(f"✓ {len(sequences_objets)} objets SequenceItem créés\n")

# Afficher les informations de chaque objet
for seq_obj in sequences_objets:
    print(f"Séquence : {seq_obj.identifiant}")
    print(f"  Longueur : {seq_obj.length()} nucléotides")
    print(f"  %GC : {seq_obj.gc_content():.2f}%")
    print(f"  Premiers 30 bp : {str(seq_obj.sequence)[:30]}...")
    print("-" * 40)

# Exemple 3 : Tester le setter
print("\n--- Test du setter ---\n")
seq1.sequence = "AAATTTGGGCCC"
print(f"Nouvelle séquence : {seq1.sequence}")
print(f"Nouvelle longueur : {seq1.length()}")
print(f"Nouveau %GC : {seq1.gc_content():.2f}%")


--- Création d'objets SequenceItem ---

Objet créé : SequenceItem(id=seq_test, length=20)
  Identifiant : seq_test
  Description : Séquence de test Homo sapiens
  Longueur : 20 nucléotides
  %GC : 60.00%
  Complément inverse : GCGCGCTAGCTAGCTAGCAT
----------------------------------------

--- Chargement depuis FASTA ---

✓ 2 objets SequenceItem créés

Séquence : seq1
  Longueur : 120 nucléotides
  %GC : 50.00%
  Premiers 30 bp : ATGCTAGCTAGCTAGCTAGCTAGCTAGCTA...
----------------------------------------
Séquence : seq2
  Longueur : 120 nucléotides
  %GC : 50.00%
  Premiers 30 bp : ATGCTAGCTGGCTAGATAGCTAGCTTGCAT...
----------------------------------------

--- Test du setter ---

Nouvelle séquence : AAATTTGGGCCC
Nouvelle longueur : 12
Nouveau %GC : 50.00%


ÉTAPE 9 — Classe SequenceDataset

In [19]:
class SequenceDataset:
    """Classe pour gérer un ensemble de séquences biologiques."""
    
    def __init__(self):
        """Initialise un dataset vide."""
        self._items = []
    
    # Question 13 : Chargement depuis FASTA
    def load_from_fasta(self, filepath):
        """
        Charge les séquences depuis un fichier FASTA.
        
        Args:
            filepath (str): Chemin vers le fichier FASTA
        """
        self._items = []
        for record in SeqIO.parse(filepath, "fasta"):
            item = SequenceItem(
                identifiant=record.id,
                description=record.description,
                sequence=record.seq
            )
            self._items.append(item)
    
    # Question 14 : Méthodes d'analyse du dataset
    def get_longest_sequence(self):
        """Retourne la séquence la plus longue du dataset."""
        if not self._items:
            return None
        return max(self._items, key=lambda x: x.length())
    
    def average_gc_content(self):
        """Calcule le pourcentage GC moyen du dataset."""
        if not self._items:
            return 0
        total_gc = sum(item.gc_content() for item in self._items)
        return total_gc / len(self._items)
    
    # Méthodes utiles supplémentaires
    def get_shortest_sequence(self):
        """Retourne la séquence la plus courte du dataset."""
        if not self._items:
            return None
        return min(self._items, key=lambda x: x.length())
    
    def filter_by_length(self, min_length):
        """
        Filtre les séquences par longueur minimale.
        
        Args:
            min_length (int): Longueur minimale
        
        Returns:
            list: Liste des séquences valides
        """
        return [item for item in self._items if item.length() >= min_length]
    
    def get_statistics(self):
        """Retourne un résumé statistique du dataset."""
        if not self._items:
            return {}
        
        lengths = [item.length() for item in self._items]
        gc_values = [item.gc_content() for item in self._items]
        
        return {
            "nombre_sequences": len(self._items),
            "longueur_moyenne": sum(lengths) / len(lengths),
            "longueur_min": min(lengths),
            "longueur_max": max(lengths),
            "gc_moyen": sum(gc_values) / len(gc_values)
        }
    
    # Méthodes spéciales Python
    def __len__(self):
        """Retourne le nombre de séquences dans le dataset."""
        return len(self._items)
    
    def __getitem__(self, index):
        """Permet d'accéder aux séquences par index (dataset[0])."""
        return self._items[index]
    
    def __iter__(self):
        """Permet d'itérer sur le dataset (for seq in dataset)."""
        return iter(self._items)
    
    def __str__(self):
        """Représentation en chaîne du dataset."""
        return f"SequenceDataset({len(self._items)} séquences)"


# =============================================================================
# UTILISATION DE LA CLASSE SequenceDataset
# =============================================================================
print("\n--- Création et chargement du dataset ---\n")

# Créer un dataset
dataset = SequenceDataset()

# Charger depuis un fichier FASTA
fasta_path = "data/example.fasta"
dataset.load_from_fasta(fasta_path)

print(f"✓ Dataset créé : {dataset}")
print(f"  Nombre de séquences : {len(dataset)}")
print("-" * 60)

# Question 14 : Séquence la plus longue
print("\n--- Séquence la plus longue ---\n")
plus_longue = dataset.get_longest_sequence()
if plus_longue:
    print(f"Identifiant : {plus_longue.identifiant}")
    print(f"Longueur : {plus_longue.length()} nucléotides")
    print(f"%GC : {plus_longue.gc_content():.2f}%")
print("-" * 60)

# Question 14 : %GC moyen
print("\n--- Pourcentage GC moyen ---\n")
gc_moyen = dataset.average_gc_content()
print(f"%GC moyen du dataset : {gc_moyen:.2f}%")
print("-" * 60)

# Méthodes supplémentaires
print("\n--- Séquence la plus courte ---\n")
plus_courte = dataset.get_shortest_sequence()
if plus_courte:
    print(f"Identifiant : {plus_courte.identifiant}")
    print(f"Longueur : {plus_courte.length()} nucléotides")
print("-" * 60)

# Statistiques globales
print("\n--- Statistiques du dataset ---\n")
stats = dataset.get_statistics()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key} : {value:.2f}")
    else:
        print(f"{key} : {value}")
print("-" * 60)

# Filtrage par longueur
print("\n--- Filtrage (longueur ≥ 100) ---\n")
sequences_valides = dataset.filter_by_length(100)
print(f"Séquences valides : {len(sequences_valides)}")
for seq in sequences_valides:
    print(f"  - {seq.identifiant} : {seq.length()} bp")
print("-" * 60)

# Utilisation des méthodes spéciales
print("\n--- Accès aux séquences ---\n")

# Accès par index
print(f"Première séquence : {dataset[0].identifiant}")
print(f"Deuxième séquence : {dataset[1].identifiant}")

# Itération
print("\nItération sur toutes les séquences :")
for i, seq in enumerate(dataset, 1):
    print(f"  {i}. {seq.identifiant} - {seq.length()} bp - {seq.gc_content():.2f}% GC")


--- Création et chargement du dataset ---

✓ Dataset créé : SequenceDataset(2 séquences)
  Nombre de séquences : 2
------------------------------------------------------------

--- Séquence la plus longue ---

Identifiant : seq1
Longueur : 120 nucléotides
%GC : 50.00%
------------------------------------------------------------

--- Pourcentage GC moyen ---

%GC moyen du dataset : 50.00%
------------------------------------------------------------

--- Séquence la plus courte ---

Identifiant : seq1
Longueur : 120 nucléotides
------------------------------------------------------------

--- Statistiques du dataset ---

nombre_sequences : 2
longueur_moyenne : 120.00
longueur_min : 120
longueur_max : 120
gc_moyen : 50.00
------------------------------------------------------------

--- Filtrage (longueur ≥ 100) ---

Séquences valides : 2
  - seq1 : 120 bp
  - seq2 : 120 bp
------------------------------------------------------------

--- Accès aux séquences ---

Première séquence : seq1

ÉTAPE 10 — Accès à NCBI avec Entrez

In [24]:
from Bio import Entrez
import time

# Question 15 : Configuration Entrez
Entrez.email = "jihaneelkhraibi15@outlook.com" 

print("\n--- Configuration NCBI ---")
print(f"Email configuré : {Entrez.email}")
print("-" * 60)


# Question 16 : Fonction de recherche de séquences
def rechercher_sequences_ncbi(espece, gene, retmax=5):
    """
    Recherche des séquences sur NCBI.
    
    Args:
        espece (str): Nom de l'espèce (ex: "Homo sapiens")
        gene (str): Nom du gène (ex: "COI", "16S")
        retmax (int): Nombre maximum de résultats (max 5 selon consignes)
    
    Returns:
        list: Liste d'identifiants NCBI
    """
    # Construire la requête
    query = f"{espece}[Organism] AND {gene}[Gene]"
    
    print(f"\nRecherche : {query}")
    print(f"Limite : {retmax} résultats")
    
    try:
        # Effectuer la recherche
        handle = Entrez.esearch(
            db="nucleotide",      # Base de données nucléotides
            term=query,           # Requête de recherche
            retmax=retmax         # Nombre max de résultats
        )
        record = Entrez.read(handle)
        handle.close()
        
        ids = record["IdList"]
        print(f"✓ {len(ids)} séquence(s) trouvée(s)")
        
        return ids
    
    except Exception as e:
        print(f"✗ Erreur lors de la recherche : {e}")
        return []


# Question 17 : Téléchargement de séquence
def telecharger_sequence(id_ncbi, output_file):
    """
    Télécharge une séquence NCBI et la sauvegarde en FASTA.
    
    Args:
        id_ncbi (str): Identifiant NCBI
        output_file (str): Nom du fichier de sortie
    
    Returns:
        bool: True si succès, False sinon
    """
    try:
        print(f"\nTéléchargement de la séquence {id_ncbi}...")
        
        # Télécharger la séquence
        handle = Entrez.efetch(
            db="nucleotide",
            id=id_ncbi,
            rettype="fasta",     # Format FASTA
            retmode="text"       # Mode texte
        )
        
        # Sauvegarder dans un fichier
        with open(output_file, "w") as f:
            f.write(handle.read())
        handle.close()
        
        print(f"✓ Séquence sauvegardée dans {output_file}")
        return True
    
    except Exception as e:
        print(f"✗ Erreur lors du téléchargement : {e}")
        return False


def telecharger_sequences_multiples(especes, gene, output_dir="data"):
    """
    Télécharge une séquence par espèce (contrainte du projet : 1 seq/espèce).
    
    Args:
        especes (list): Liste de noms d'espèces
        gene (str): Nom du gène
        output_dir (str): Dossier de sortie
    
    Returns:
        dict: Dictionnaire {espèce: fichier}
    """
    import os
    
    # Créer le dossier si nécessaire
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    sequences_telechargees = {}
    
    for espece in especes:
        print(f"\n{'='*60}")
        print(f"Espèce : {espece}")
        print(f"{'='*60}")
        
        # Rechercher
        ids = rechercher_sequences_ncbi(espece, gene, retmax=5)
        
        if ids:
            # Prendre seulement la première séquence (contrainte : 1 par espèce)
            id_choisi = ids[0]
            
            # Nom du fichier
            espece_clean = espece.replace(" ", "_")
            output_file = f"{output_dir}/{espece_clean}_{gene}.fasta"
            
            # Télécharger
            if telecharger_sequence(id_choisi, output_file):
                sequences_telechargees[espece] = output_file
            
            # Pause pour respecter les limites de NCBI (max 3 requêtes/sec)
            time.sleep(0.4)
        else:
            print(f"✗ Aucune séquence trouvée pour {espece}")
    
    return sequences_telechargees


# =============================================================================
# UTILISATION
# =============================================================================

# Contraintes du projet : 5-10 espèces
especes = [
    "Homo sapiens",
    "Mus musculus",
    "Canis lupus",
    "Felis catus",
    "Pan troglodytes"
]

# Gène à rechercher (exemple : COI/COX1)
gene = "COI"

print("\n--- Recherche simple ---")
# Test de recherche pour une espèce
ids = rechercher_sequences_ncbi("Homo sapiens", "COI", retmax=3)
print(f"IDs trouvés : {ids}")

print("\n--- Téléchargement d'une séquence ---")
# Test de téléchargement si on a trouvé des IDs
if ids:
    telecharger_sequence(ids[0], "data/test_sequence.fasta")

print("\n--- Téléchargement multiple (1 séquence par espèce) ---")
# Télécharger une séquence pour chaque espèce
resultats = telecharger_sequences_multiples(especes, gene, output_dir="data")

print("\n" + "="*60)
print("RÉSUMÉ DES TÉLÉCHARGEMENTS")
print("="*60)
for espece, fichier in resultats.items():
    print(f"✓ {espece} → {fichier}")

print(f"\nTotal : {len(resultats)} séquences téléchargées")


--- Configuration NCBI ---
Email configuré : jihaneelkhraibi15@outlook.com
------------------------------------------------------------

--- Recherche simple ---

Recherche : Homo sapiens[Organism] AND COI[Gene]
Limite : 3 résultats
✓ 3 séquence(s) trouvée(s)
IDs trouvés : ['2579155578', '251831106', '2234442025']

--- Téléchargement d'une séquence ---

Téléchargement de la séquence 2579155578...
✓ Séquence sauvegardée dans data/test_sequence.fasta

--- Téléchargement multiple (1 séquence par espèce) ---

Espèce : Homo sapiens

Recherche : Homo sapiens[Organism] AND COI[Gene]
Limite : 5 résultats
✓ 5 séquence(s) trouvée(s)

Téléchargement de la séquence 2579155578...
✓ Séquence sauvegardée dans data/Homo_sapiens_COI.fasta

Espèce : Mus musculus

Recherche : Mus musculus[Organism] AND COI[Gene]
Limite : 5 résultats
✓ 5 séquence(s) trouvée(s)

Téléchargement de la séquence 2327616453...
✓ Séquence sauvegardée dans data/Mus_musculus_COI.fasta

Espèce : Canis lupus

Recherche : Canis lupu

In [25]:
def fusionner_sequences_fasta(fichiers, output_file="ncbi_sequences.fasta"):
    """Fusionne plusieurs fichiers FASTA en un seul."""
    all_records = []
    
    for fichier in fichiers:
        try:
            records = list(SeqIO.parse(fichier, "fasta"))
            all_records.extend(records)
            print(f"  ✓ Ajouté : {fichier} ({len(records)} séquence)")
        except Exception as e:
            print(f"  ✗ Erreur avec {fichier} : {e}")
    
    # Sauvegarder toutes les séquences dans un fichier
    SeqIO.write(all_records, output_file, "fasta")
    print(f"\n✓ {len(all_records)} séquences fusionnées dans {output_file}")

# Utilisation après le téléchargement
print("\n--- Fusion des séquences ---")
fichiers = list(resultats.values())
fusionner_sequences_fasta(fichiers, "ncbi_sequences.fasta")


--- Fusion des séquences ---
  ✓ Ajouté : data/Homo_sapiens_COI.fasta (1 séquence)
  ✓ Ajouté : data/Mus_musculus_COI.fasta (1 séquence)
  ✓ Ajouté : data/Canis_lupus_COI.fasta (1 séquence)
  ✓ Ajouté : data/Felis_catus_COI.fasta (1 séquence)
  ✓ Ajouté : data/Pan_troglodytes_COI.fasta (1 séquence)

✓ 5 séquences fusionnées dans ncbi_sequences.fasta


ÉTAPE 11 — Alignement pairwise

In [5]:

from Bio.Seq import Seq
from Bio import Align

# =============================================================================
# FONCTIONS D'ALIGNEMENT
# =============================================================================

def aligner_sequences(reference, sequences):
    """
    Aligne des séquences contre une référence (VERSION SIMPLIFIÉE).
    
    Args:
        reference (Seq ou str): Séquence de référence
        sequences (list): Liste d'objets SequenceItem
    
    Returns:
        list: Liste de dictionnaires avec id, score, sequence
    """
    resultats = []
    
    # Créer l'aligneur
    aligner = Align.PairwiseAligner()
    aligner.mode = 'global'
    aligner.match_score = 1
    aligner.mismatch_score = 0
    aligner.gap_score = 0
    
    ref_str = str(reference)
    
    print("\nAlignements en cours...")
    for seq_item in sequences:
        seq_str = str(seq_item.sequence)
        
        try:
            # Réaliser l'alignement et récupérer DIRECTEMENT le score
            alignments = aligner.align(ref_str, seq_str)
            score = alignments.score
            
            resultats.append({
                "id": seq_item.identifiant,
                "score": score,
                "sequence": seq_item
            })
            
            print(f"  ✓ {seq_item.identifiant} : score = {score:.0f}")
            
        except Exception as e:
            print(f"  ✗ Erreur avec {seq_item.identifiant} : {e}")
    
    # Trier par score décroissant
    resultats.sort(key=lambda x: x["score"], reverse=True)
    
    return resultats


def calculer_similarite_simple(reference, sequences):
    """
    Calcule la similarité basée sur des k-mers (plus rapide et sans overflow).
    
    Args:
        reference (Seq ou str): Séquence de référence
        sequences (list): Liste d'objets SequenceItem
    
    Returns:
        list: Liste de dictionnaires avec id, score, sequence
    """
    resultats = []
    ref_str = str(reference).upper()
    
    # Utiliser des k-mers pour la similarité
    k = 10  # Taille des k-mers
    ref_kmers = set(ref_str[i:i+k] for i in range(len(ref_str)-k+1))
    
    print("\nCalcul de similarité (k-mers)...")
    for seq_item in sequences:
        seq_str = str(seq_item.sequence).upper()
        
        # Créer les k-mers de la séquence
        seq_kmers = set(seq_str[i:i+k] for i in range(len(seq_str)-k+1))
        
        # Calculer la similarité de Jaccard
        if len(ref_kmers) > 0 and len(seq_kmers) > 0:
            intersection = len(ref_kmers & seq_kmers)
            union = len(ref_kmers | seq_kmers)
            similarite = (intersection / union) * 100
        else:
            similarite = 0
        
        # Score proportionnel à la similarité
        score = similarite * max(len(ref_str), len(seq_str)) / 100
        
        resultats.append({
            "id": seq_item.identifiant,
            "score": score,
            "sequence": seq_item
        })
        
        print(f"  ✓ {seq_item.identifiant} : score = {score:.0f} (similarité = {similarite:.2f}%)")
    
    # Trier par score décroissant
    resultats.sort(key=lambda x: x["score"], reverse=True)
    
    return resultats




# Charger les séquences
print("\n--- Chargement des séquences ---")
fasta_path = "ncbi_sequences.fasta"

dataset = SequenceDataset()
dataset.load_from_fasta(fasta_path)

print(f"✓ {len(dataset)} séquences chargées")
print("-" * 60)

# Référence
print("\n--- Sélection de la référence ---")
reference_seq = dataset[0]
print(f"Référence : {reference_seq.identifiant}")
print(f"  Longueur : {reference_seq.length()} bp")
print("-" * 60)

# Préparer les séquences à comparer
sequences_a_comparer = [dataset[i] for i in range(1, len(dataset))]

print(f"\n--- Alignement contre {len(sequences_a_comparer)} séquence(s) ---")

# Similarité k-mers (RECOMMANDÉ - pas d'overflow)
resultats_alignement = calculer_similarite_simple(reference_seq.sequence, sequences_a_comparer)

# Résultats
print("\n--- Résultats (triés par similarité décroissante) ---")
for i, r in enumerate(resultats_alignement, 1):
    seq = r["sequence"]
    score = r["score"]
    print(f"{i}. {seq.identifiant}")
    print(f"   Score : {score:.0f}")
    print(f"   Longueur : {seq.length()} bp")
    print(f"   %GC : {seq.gc_content():.2f}%")
    print("-" * 40)


--- Chargement des séquences ---
✓ 5 séquences chargées
------------------------------------------------------------

--- Sélection de la référence ---
Référence : OR327029.1
  Longueur : 16567 bp
------------------------------------------------------------

--- Alignement contre 4 séquence(s) ---

Calcul de similarité (k-mers)...
  ✓ OM948981.1 : score = 1022 (similarité = 6.17%)
  ✓ MZ099455.1 : score = 58 (similarité = 0.35%)
  ✓ NC_001700.1 : score = 1187 (similarité = 6.98%)
  ✓ OZ347562.1 : score = 241 (similarité = 1.45%)

--- Résultats (triés par similarité décroissante) ---
1. NC_001700.1
   Score : 1187
   Longueur : 17009 bp
   %GC : 40.33%
----------------------------------------
2. OM948981.1
   Score : 1022
   Longueur : 16300 bp
   %GC : 36.75%
----------------------------------------
3. OZ347562.1
   Score : 241
   Longueur : 657 bp
   %GC : 49.01%
----------------------------------------
4. MZ099455.1
   Score : 58
   Longueur : 658 bp
   %GC : 41.03%
----------------

ÉTAPE 12 — Export des résultats en CSV

In [6]:

import csv

def exporter_resultats_csv(resultats, output_file="results.csv"):
    """
    Exporte les résultats dans un fichier CSV.
    
    Args:
        resultats (list): Liste des résultats d'alignement
        output_file (str): Nom du fichier CSV
    """
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        # En-tête du CSV
        writer.writerow(["ID", "Longueur", "GC%", "Score_Similarite"])
        
        # Écrire les données
        for r in resultats:
            seq = r["sequence"]
            writer.writerow([
                seq.identifiant,
                seq.length(),
                round(seq.gc_content(), 2),
                round(r["score"], 2)
            ])
    
    print(f"✓ Résultats exportés dans {output_file}")


# UTILISATION
exporter_resultats_csv(resultats_alignement, "results.csv")

# Afficher le contenu du CSV
print("\n--- Aperçu du fichier results.csv ---")
with open("results.csv", 'r', encoding='utf-8') as f:
    print(f.read())

print("-" * 60)
print("✅ Fichier results.csv créé avec succès !")

✓ Résultats exportés dans results.csv

--- Aperçu du fichier results.csv ---
ID,Longueur,GC%,Score_Similarite
NC_001700.1,17009,40.33,1187.04
OM948981.1,16300,36.75,1022.4
OZ347562.1,657,49.01,240.51
MZ099455.1,658,41.03,58.48

------------------------------------------------------------
✅ Fichier results.csv créé avec succès !


ÉTAPE 13 — Test complet du projet

In [7]:
import os

def test_complet(fasta_path="ncbi_sequences.fasta"):
    """Programme principal - test de toutes les fonctionnalités."""
    
    print("\n🔬 BIOSEQ ANALYZER - ANALYSE COMPLÈTE")
    print("="*70)
    
    # [1/7] CHARGEMENT
    print("\n[1/7] Chargement des séquences...")
    dataset = SequenceDataset()
    dataset.load_from_fasta(fasta_path)
    print(f"      ✓ {len(dataset)} séquences chargées")
    
    # [2/7] VALIDATION
    print("\n[2/7] Validation des séquences...")
    valides = [seq for seq in dataset if seq.length() >= 100]
    invalides = [seq for seq in dataset if seq.length() < 100]
    print(f"      ✓ Séquences valides (≥100 bp) : {len(valides)}")
    print(f"      ✓ Séquences invalides (<100 bp) : {len(invalides)}")
    
    # [3/7] OBJETS
    print("\n[3/7] Vérification des objets...")
    print(f"      ✓ Type : {type(dataset[0]).__name__}")
    print(f"      ✓ Encapsulation POO : vérifiée")
    
    # [4/7] STATISTIQUES
    print("\n[4/7] Statistiques...")
    longueurs = [seq.length() for seq in dataset]
    gc_values = [seq.gc_content() for seq in dataset]
    
    print(f"      ✓ Longueur moyenne : {sum(longueurs)/len(longueurs):.0f} bp")
    print(f"      ✓ Longueur min : {min(longueurs)} bp")
    print(f"      ✓ Longueur max : {max(longueurs)} bp")
    print(f"      ✓ %GC moyen : {sum(gc_values)/len(gc_values):.2f}%")
    
    # [5/7] ALIGNEMENTS
    print("\n[5/7] Alignements...")
    if len(dataset) > 1:
        reference = dataset[0]
        sequences_a_aligner = [dataset[i] for i in range(1, len(dataset))]
        resultats = calculer_similarite_simple(reference.sequence, sequences_a_aligner)
        print(f"      ✓ {len(resultats)} alignements effectués")
    else:
        resultats = []
    
    # [6/7] EXPORT CSV
    print("\n[6/7] Export CSV...")
    if resultats:
        exporter_resultats_csv(resultats, "results.csv")
    
    # [7/7] VÉRIFICATION
    print("\n[7/7] Vérification finale...")
    
    fonctionnalites = {
        "Lecture FASTA": True,
        "Manipulation séquences": True,
        "Validation": len(valides) > 0,
        "Classe SequenceItem": True,
        "Classe SequenceDataset": True,
        "Statistiques": True,
        "Alignement": len(resultats) > 0 if len(dataset) > 1 else True,
        "Export CSV": os.path.exists("results.csv")
    }
    
    for fonc, status in fonctionnalites.items():
        print(f"      {'✓' if status else '✗'} {fonc}")
    
    # RÉSUMÉ
    print("\n" + "="*70)
    print("✅ ANALYSE TERMINÉE AVEC SUCCÈS !")
    print("="*70)
    
    print("\n📊 RÉSUMÉ :")
    print(f"   • Séquences : {len(dataset)}")
    print(f"   • Longueur moyenne : {sum(longueurs)/len(longueurs):.0f} bp")
    print(f"   • %GC moyen : {sum(gc_values)/len(gc_values):.2f}%")
    print(f"   • Alignements : {len(resultats)}")
    
    print(f"\n📁 FICHIERS :")
    print(f"   • {fasta_path}")
    print(f"   • results.csv")
    
    print(f"\n🎓 TOUTES LES QUESTIONS (1-20) COMPLÉTÉES !")
    print("="*70)
    
    return dataset, resultats


# EXÉCUTION
print("\n🚀 LANCEMENT DU TEST COMPLET")
dataset_final, resultats_final = test_complet("ncbi_sequences.fasta")


ÉTAPE 15 — TEST COMPLET DU PROJET

🚀 LANCEMENT DU TEST COMPLET

🔬 BIOSEQ ANALYZER - ANALYSE COMPLÈTE

[1/7] Chargement des séquences...
      ✓ 5 séquences chargées

[2/7] Validation des séquences...
      ✓ Séquences valides (≥100 bp) : 5
      ✓ Séquences invalides (<100 bp) : 0

[3/7] Vérification des objets...
      ✓ Type : SequenceItem
      ✓ Encapsulation POO : vérifiée

[4/7] Statistiques...
      ✓ Longueur moyenne : 10238 bp
      ✓ Longueur min : 657 bp
      ✓ Longueur max : 17009 bp
      ✓ %GC moyen : 42.31%

[5/7] Alignements...

Calcul de similarité (k-mers)...
  ✓ OM948981.1 : score = 1022 (similarité = 6.17%)
  ✓ MZ099455.1 : score = 58 (similarité = 0.35%)
  ✓ NC_001700.1 : score = 1187 (similarité = 6.98%)
  ✓ OZ347562.1 : score = 241 (similarité = 1.45%)
      ✓ 4 alignements effectués

[6/7] Export CSV...
✓ Résultats exportés dans results.csv

[7/7] Vérification finale...
      ✓ Lecture FASTA
      ✓ Manipulation séquences
      ✓ Validation
      ✓ Classe Sequ